In [64]:
import numpy as np
import pandas as pd
import os
import re
import matplotlib.pyplot as plt
import networkx as nx
"""
# Set display options to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
"""
import warnings
warnings.filterwarnings('ignore')

# Reset display options to default
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')
pd.reset_option('display.width')
pd.reset_option('display.max_colwidth')


In [65]:
test_df=pd.read_csv('CSVs/test.csv',delimiter="\t")
test_df.drop(test_df[test_df['Correct First Attempt'].isna()].index,inplace=True)
test_df.drop(columns=['Step Start Time',
                         "Step End Time","Step Duration (sec)",
                        "Error Step Duration (sec)",
                         "Hints","Incorrects","Corrects",
                         "Correct First Attempt",'Row'])

,Anon Student Id,Problem Hierarchy,Problem Name,Problem View,Step Name,First Transaction Time,Correct Transaction Time,Correct Step Duration (sec),KC(Default),Opportunity(Default)
14,8o0mJnIpFH,"Unit CTA1_06, Section CTA1_06-3",JAN05,2,R5C1,NaN,NaN,NaN,Using simple numbers~~Using large numbers~~Fin...,24~~15~~18
15,8o0mJnIpFH,"Unit ES_03, Section ES_03-6",EG51A,2,constant termsR2,NaN,NaN,NaN,NaN,NaN
16,8o0mJnIpFH,"Unit CTA1_08, Section CTA1_08-2",FEB03,1,yMin,NaN,NaN,NaN,Changing axis bounds,98
17,8o0mJnIpFH,"Unit ES_04, Section ES_04-15",LIT58,1,7+y*(b+n) = r+f,NaN,NaN,NaN,[SkillRule: Eliminate Parens; {CLT nested; CLT...,70
18,8o0mJnIpFH,"Unit CTA1_10, Section CTA1_10-5",DISTFB08_SP,1,R3C2,NaN,NaN,NaN,"Using simple numbers~~Write expression, positi...",78~~45~~56
...,...,...,...,...,...,...,...,...,...,...
1135,C0qLCIhI0O,"Unit CTA1_24, Section CTA1_24-1",L5FB05,1,R4C2,NaN,NaN,NaN,Entering a given,1
1136,e95f4UtF4I,"Unit EXPT-PRODUCT-SIMP-A_ES, Section EXPT-PROD...",EG-EPS01-FIXED,1,FinalAnswer,NaN,NaN,NaN,perform-mult-sp,11
1137,e95f4UtF4I,"Unit EXPT-QUOTIENT-SIMP-A_ES, Section EXPT-QUO...",EG-EQS08,1,2/1,NaN,NaN,NaN,NaN,NaN
1138,e95f4UtF4I,"Unit CTA1_24, Section CTA1_24-1",PERCENT07,1,R6C2,NaN,NaN,NaN,Entering a given,6


In [66]:
test_df[['Unit', 'Section']] = test_df['Problem Hierarchy'].str.split(', ', expand=True)
# Remove the words "Unit " and "Section " from the new columns
test_df['Unit'] = test_df['Unit'].str.replace('Unit ', '', regex=False)
test_df['Section'] = test_df['Section'].str.replace('Section ', '', regex=False)
test_df['KC(Default)'] = test_df['KC(Default)'].fillna('nan')
test_df['Opportunity(Default)'] = test_df['Opportunity(Default)'].fillna('nan')

In [67]:
def clean_skill(skill):
    # Patrón para buscar texto entre los primeros dos puntos y después de los primeros puntos y comas
    if str(skill)[0]=='[':
        if "SkillRule: done no solutions" in skill:
            return "done no solutions"
        else:
            skill=skill[skill.index(' ')+1:]
            skill=skill[:skill.index(';')]
            return skill
    
    else:
        return skill

In [68]:
test_df['KC(Default)']=test_df['KC(Default)'].apply(clean_skill)

#Reemplazar la columna problem Hierarchy por unit y section

dfa=test_df.drop(columns=['Problem Hierarchy'])
test_df=dfa.reindex(columns=['Row', 'Anon Student Id', 'Unit',
       'Section', 'Problem Name',
       'Problem View', 'Step Name', 'Step Start Time',
       'First Transaction Time', 'Correct Transaction Time', 'Step End Time',
       'Step Duration (sec)', 'Correct Step Duration (sec)',
       'Error Step Duration (sec)', 'Correct First Attempt', 'Incorrects',
       'Hints', 'Corrects', 'KC(Default)', 'Opportunity(Default)'])

In [69]:
with open('No validos.txt',"r") as file:
        content = file.read()

# Convertir el contenido en una lista de Python
data_list = content.strip().split(',')

# Eliminar espacios en blanco adicionales y comillas
data_list = [item.strip().strip("'").strip('"') for item in data_list]

print(data_list)

['LIT48', 'EG36', 'LIT44', 'LIT88', 'LIT38', 'LIT39', 'LIT86', 'LIT84', 'LIT89', 'LIT87', '2PTFB16', 'YGT-2X', 'SYS07', 'SYSFB06', 'NEWSQUARE2TRE', 'EG-FACTOR20', 'EG-FACTOR06', 'EG-FACTOR08', 'EG-FACTOR09', 'EG-FACTOR10', 'EG-FACTOR16', 'EG-FACTOR17', 'EG-FACTOR07', 'EG-FACTOR02', 'EG-S-FACTOR23', 'EG-S-FACTOR28', 'EG-S-FACTOR30', 'EG-S-FACTOR22', 'EG-S-FACTOR29', 'EG-S-FACTOR10', 'EG-S-FACTOR09', 'EG-S-FACTOR20', 'EG-S-FACTOR12', 'EG-S-FACTOR04', 'EG-S-FACTOR19', 'EG-S-FACTOR06', 'FBSMR2', 'EG-S-FACTOR15', 'EG-S-FACTOR17', 'EG-S-FACTOR08', 'EG-S-FACTOR18', 'EG-S-FACTOR13', '2PTFB22', 'IPCSNOW', 'IPCPAIGE', '1PTFB10', 'BH1T09', 'BH1T12', 'BH1T14', 'BH1T25', 'NOV09', 'NOV11', 'NOV06', 'PROP17', 'SYSFB13', 'SYSFB08', 'SYSFB19', 'SYS19', 'EG6-FIXED', 'EG63A', 'EG63', 'PERCENT15', 'FOR01', 'BH1T50', 'PERCENT09', 'REAL36', 'PROP11', 'DEC07', 'L3JAN02', 'BH1T03', 'BH1T38', 'REAL25', 'L5FB17', 'BH1T06', 'PROP19', 'DIST01', 'PROP18', 'REAL16', 'L3REAL12', 'PROP14', 'TRANSFORMV001', 'TRANSFORM

In [70]:
test_df = test_df[~test_df['Problem Name'].isin(data_list)]
test_df

,Row,Anon Student Id,Unit,Section,Problem Name,Problem View,Step Name,Step Start Time,First Transaction Time,Correct Transaction Time,Step End Time,Step Duration (sec),Correct Step Duration (sec),Error Step Duration (sec),Correct First Attempt,Incorrects,Hints,Corrects,KC(Default),Opportunity(Default)
14,21616,8o0mJnIpFH,CTA1_06,CTA1_06-3,JAN05,2,R5C1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,Using simple numbers~~Using large numbers~~Fin...,24~~15~~18
15,22178,8o0mJnIpFH,ES_03,ES_03-6,EG51A,2,constant termsR2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,nan,nan
16,22320,8o0mJnIpFH,CTA1_08,CTA1_08-2,FEB03,1,yMin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,Changing axis bounds,98
17,22862,8o0mJnIpFH,ES_04,ES_04-15,LIT58,1,7+y*(b+n) = r+f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,Eliminate Parens,70
18,23277,8o0mJnIpFH,CTA1_10,CTA1_10-5,DISTFB08_SP,1,R3C2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,"Using simple numbers~~Write expression, positi...",78~~45~~56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1135,1075314,C0qLCIhI0O,CTA1_24,CTA1_24-1,L5FB05,1,R4C2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,Entering a given,1
1136,1076841,e95f4UtF4I,EXPT-PRODUCT-SIMP-A_ES,EXPT-PRODUCT-SIMPLIFYING-A_ES-3,EG-EPS01-FIXED,1,FinalAnswer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,perform-mult-sp,11
1137,1076862,e95f4UtF4I,EXPT-QUOTIENT-SIMP-A_ES,EXPT-QUOTIENT-SIMPLIFYING-A_ES-2,EG-EQS08,1,2/1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,nan,nan
1138,1076891,e95f4UtF4I,CTA1_24,CTA1_24-1,PERCENT07,1,R6C2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,Entering a given,6


In [71]:
rendimiento_df=pd.read_csv('CSVs/Rendimiento.csv')
df_dificultad=pd.read_csv('CSVs/Dificultad.csv')

In [72]:
test_df.fillna('nan',inplace=True)
df_dificultad.fillna('nan',inplace=True)
df_dificultad['KC(Default)']=df_dificultad['KC(Default)'].str.strip()
test_df.columns = test_df.columns.str.strip()
merged_test_df = pd.merge(test_df, df_dificultad, left_on=["Problem Name", "Step Name", "KC(Default)"], right_on=["Problem Name", "Step Name", "KC(Default)"], how="left")
merged_test_df.drop_duplicates(subset=['Row'],keep='first',inplace=True)
merged_test_df

,Row,Anon Student Id,Unit,Section,Problem Name,Problem View,Step Name,Step Start Time,First Transaction Time,Correct Transaction Time,...,Error Step Duration (sec),Correct First Attempt,Incorrects,Hints,Corrects,KC(Default),Opportunity(Default),E Conceptual,E Resolución,Dificultad
0,21616,8o0mJnIpFH,CTA1_06,CTA1_06-3,JAN05,2,R5C1,nan,nan,nan,...,nan,0.0,nan,nan,nan,Using simple numbers~~Using large numbers~~Fin...,24~~15~~18,0.5176,0.2518,8.0
1,22178,8o0mJnIpFH,ES_03,ES_03-6,EG51A,2,constant termsR2,nan,nan,nan,...,nan,1.0,nan,nan,nan,nan,nan,0.0661,0.0000,1.0
2,22320,8o0mJnIpFH,CTA1_08,CTA1_08-2,FEB03,1,yMin,nan,nan,nan,...,nan,1.0,nan,nan,nan,Changing axis bounds,98,0.0859,0.0000,1.0
3,22862,8o0mJnIpFH,ES_04,ES_04-15,LIT58,1,7+y*(b+n) = r+f,nan,nan,nan,...,nan,1.0,nan,nan,nan,Eliminate Parens,70,NaN,NaN,NaN
4,23277,8o0mJnIpFH,CTA1_10,CTA1_10-5,DISTFB08_SP,1,R3C2,nan,nan,nan,...,nan,0.0,nan,nan,nan,"Using simple numbers~~Write expression, positi...",78~~45~~56,0.2898,0.5796,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
497,1075314,C0qLCIhI0O,CTA1_24,CTA1_24-1,L5FB05,1,R4C2,nan,nan,nan,...,nan,1.0,nan,nan,nan,Entering a given,1,0.0625,0.0000,1.0
498,1076841,e95f4UtF4I,EXPT-PRODUCT-SIMP-A_ES,EXPT-PRODUCT-SIMPLIFYING-A_ES-3,EG-EPS01-FIXED,1,FinalAnswer,nan,nan,nan,...,nan,1.0,nan,nan,nan,perform-mult-sp,11,0.1000,0.0000,1.0
499,1076862,e95f4UtF4I,EXPT-QUOTIENT-SIMP-A_ES,EXPT-QUOTIENT-SIMPLIFYING-A_ES-2,EG-EQS08,1,2/1,nan,nan,nan,...,nan,1.0,nan,nan,nan,nan,nan,NaN,NaN,NaN
500,1076891,e95f4UtF4I,CTA1_24,CTA1_24-1,PERCENT07,1,R6C2,nan,nan,nan,...,nan,1.0,nan,nan,nan,Entering a given,6,0.0000,0.0000,1.0


In [73]:
merged_test_df2 = pd.merge(merged_test_df, rendimiento_df, 
                                 left_on=["Anon Student Id"], 
                                 right_on=["Anon Student Id"], 
                                 how="left")

In [74]:
merged_test_df2.dropna(subset=['Rendimiento','Dificultad'],inplace=True)


In [75]:
# Paso 1: Dividir las celdas en listas usando el delimitador '~~'
merged_test_df2['KC(Default)'] = merged_test_df2['KC(Default)'].apply(lambda x: x.split('~~') if x != 'nan' else [])
merged_test_df2['Opportunity(Default)'] = merged_test_df2['Opportunity(Default)'].apply(lambda x: x.split('~~') if x != 'nan' else [])

# Número máximo de skills y oportunidades
max_kc_len = 5
max_opportunity_len = 5

# Paso 2: Expandir en columnas separadas y rellenar con ceros
for i in range(max_kc_len):
    merged_test_df2[f'KC_{i+1}'] = merged_test_df2['KC(Default)'].apply(lambda x: x[i] if i < len(x) else 0)

for i in range(max_opportunity_len):
    merged_test_df2[f'Opportunity_{i+1}'] = merged_test_df2['Opportunity(Default)'].apply(lambda x: x[i] if i < len(x) else 0)

# Eliminar columnas originales si no son necesarias
merged_test_df2.drop(columns=['KC(Default)', 'Opportunity(Default)'], inplace=True)

In [76]:
merged_test_df_final=merged_test_df2.drop(columns=['E Conceptual','E Resolución',
                         'X','Y','Step Start Time',
                         "Step End Time","Step Duration (sec)",
                        "Error Step Duration (sec)",
                         "Hints","Incorrects","Corrects",'Row'
                         ],inplace=True)

In [77]:
df_to_stack=pd.read_csv("CSVs/Test_to_merge.csv")

In [78]:
final_df_test=pd.concat([merged_test_df_final,df_to_stack], ignore_index=True)

In [79]:
final_df_test.to_csv("CSVs/Clean_test.csv")